# Analysers in Intel® Sytem Studio

### Profiling a Computer Vision Application

OpenCV in Intel® Distribution of OpenVINO™ leverages a tracing facility for efficient development of Computer Vision Applications.

Tracing framework is built-in directly into the OpenCV source code. To use it, you only need to compile and link your application with modern version of OpenCV.

OpenCV tracing relies on the ITT API, an [Intel®-provided profiling framework](https://software.intel.com/en-us/articles/intel-itt-api-open-source).

In [1]:
#The tracing is enabled by default at compile time. 
#But it's not activated by default at runtime, because it impacts the performance. 
#It's activated when the environment variable OPENCV_TRACE is set (OPENCV_TRACE=1) 
#or when OpenCV-based app is running within a specialized profiling software supporting 
#Instrumentation and Tracing Technology (ITT).

%env OPENCV_TRACE=0

env: OPENCV_TRACE=0


### Using VTune™ Amplifier

The handiest way to collect a trace is to run your application under control of Intel® VTune™ Amplifier. With VTune™ you can correlate your OpenCV activity with various important metrics for CPU and GPU. Finally, VTune™ captures most Intel APIs beyond OpenCV, for example OpenCL. So with these tools you can see when OpenCV calls are translated into OpenCL tasks.<br><br>
![alt text](VTune1.png) <br>
Just run analyzed OpenCV Application under Intel® VTune™ Amplifier with enabled "Analyze user tasks" option.

In [ ]:
!python rm-rf.py runs

In [ ]:
#VTune Hotspot analysis
!start "" cmd /K amplxe-cl \
-collect hotspots \
--result-dir runs/vtune/cpu_run_1 \
--duration 15 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#Generate VTune Hotspot analysis report
%sx amplxe-cl \
-report hotspots \
-r=runs/vtune/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#This example displays a Hotspots report for the r001hs result, 
#presenting CPU time for the functions of the target in  
#descending order starting from the most time-consuming function.

%sx amplxe-cl -report hotspots -r runs/vtune/cpu_run_1

In [ ]:
%%capture
#Launch VTune Amplifier 2019 GUI
!"C:\Program Files (x86)\IntelSWTools\VTune Amplifier 2019\bin64\amplxe-gui.exe" runs/vtune/cpu_run_1/cpu_run_1.amplxe

### Using Intel® Advisor™ Amplifier
![alt text](Advise1.png) <br>

<font color=pink><b>Command Line Interface Reference</b></font>

You can use the Intel® Advisor command line interface, advixe-cl, to run analysis and data collection. This makes it possible to automate many tasks or analyze an application running on remote hosts. The results can be viewed through the Intel Advisor GUI, or used to generate various kinds of reports.
<br><br>
NOTE
<br>
Before using the command line, you must set command line environment variables.
For instructions, see [Setting and Using Intel Advisor Environment Variables](https://software.intel.com/en-us/advisor-user-guide-setting-and-using-intel-advisor-environment-variables).

#### Survey Report Purpose and Usage

Run a Survey analysis to generate a <b>Survey Report</b> that offers integrated compiler report data and performance data for your target application all in one place. Optionally run a Trip Counts analysis and/or FLOP analysis to add data to the Survey Report. The Roofline analysis runs a Survey analysis followed by a FLOP analysis automatically.

<font color=blue><b>Survey analysis</b></font> - Identifies:
* Where vectorization, or parallelization with threads, will pay off the most
* If vectorized loops are providing benefit, and if not, why not
* Un-vectorized loops and why they are not vectorized
* Performance problems in general

In [ ]:
#Survey analysis
!start "" cmd /C advixe-cl.exe \
--collect survey \
--duration 15 \
--project-dir runs/advise/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
#Create the Survey analysis report
%sx advixe-cl.exe \
--report survey \
--project-dir runs/advise/cpu_run_1 \
--format text \
--report-output=runs/advise/cpu_run_1/cpu_run_1.txt \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"


<font color=blue><b>Trip Counts analysis</b></font> - Dynamically identifies the number of times loops and functions are invoked and executed (also called call count/loop count and iteration count respectively). Use Trip Counts data to:
* Detect loops with too-small trip counts and trip counts that are not a multiple of vector length.
* Analyze parallelism granularity more deeply.

<font color=blue><b>FLOP analysis</b></font> - Dynamically measures floating-point and integer operations, and memory traffic. Use the FLOP analysis to generate application memory usage and performance values that help you make better decisions about your vectorization strategy.

In [ ]:
#Trip counts analysis and FLOP analysis
!start "" cmd /C advixe-cl.exe \
--collect tripcounts -flop -stacks \
--project-dir runs/advise/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
!start "" cmd /C advixe-cl.exe \
--collect map –mark-up-list=1,2,3,4 \
--project-dir runs/advise/cpu_run_1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [ ]:
!xcopy "C:\Program Files (x86)\IntelSWTools\Advisor\pythonapi\examples" . /s /e

In [ ]:
#Generate a combined report showing all data collected
!"C:\Program Files (x86)\IntelSWTools\Advisor\advixe-vars.bat"
#!python "joined.py" > report.txt

In [ ]:
#Generate an html report
import subprocess
subprocess.call(["advixe-python.exe", "to_html.py runs/advise"])

In [ ]:
!advixe-python.exe to_html.py runs/advise/cpu_run_1

In [ ]:
#You need to do a memory access pattern collection to collect cache statistics
!start "" cmd /C advixe-cl.exe \
--collect map –mark-up-list=4 \
--project-dir advise1 \
-- RelWithDebInfo/monitor.exe \
-m="pedestrian-and-vehicle-detector-adas-0001.bin" \
-c="pedestrian-and-vehicle-detector-adas-0001.xml" \
-i="car-detection.mp4" \
-cc=0.7 -e="b"

In [3]:
#conda install -c blaze sqlite3
!advixe-python.exe cache.py runs/advise/cpu_run_1

In [2]:
!advixe-python.exe system_info.py runs/advise/cpu_run_1

mpi_rank: None
cpu_stepping: 10
hostname: DESKTOP-LMTEGJ7
cpu_brand_name: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
cpu_model: 142
cpu_frequency: 2700000000
instruction_set: AVX2
cpu_family: 6
physical_core_count: 4
os: windows
logical_cpu_count: 8


In [5]:
%%capture
#Launch Intel Advisor 2019 GUI
!"C:\Program Files (x86)\IntelSWTools\Advisor 2019\bin32\advixe-gui.exe"

# Using Intel® Inspector

Intel® Inspector is a dynamic memory and threading error checking tool for users developing serial and multithreaded applications on Windows* and Linux* operating systems. Follow these steps to get started using the Intel Inspector. <br><br>
![alt text](Inspector1.jpg) <br>

In [6]:
!start "" cmd /C "C:\Program Files (x86)\IntelSWTools\Inspector 2019\inspxe-vars.bat"
#For the command line interface:
!start "" cmd /C "C:\Program Files (x86)\IntelSWTools\Inspector 2019\bin64\inspxe-cl.exe"

In [7]:
#List all checking modes
!inspxe-cl.exe -collect-list

Available analysis types:
    Name  Description 
    mi1   Detect Leaks
    mi2   Detect Memory Problems
    mi3   Locate Memory Problems
    ti1   Detect Deadlocks
    ti2   Detect Deadlocks and Data Races
    ti3   Locate Deadlocks and Data Races 


In [8]:
%sx inspxe-cl -collect mi1 -r runs/inspect/mi1 mm1.exe

['Warning: Encountered an issue while loading result [C:\\Users\\NUC8i7\\Desktop\\EWDemo\\runs\\inspect\\mi1]. Run terminated abnormally. Some data may be lost. ',
 '  ',
 '1 new problem(s) found ',
 '    1 Missing allocation problem(s) detected ']

In [9]:
%sx inspxe-cl -create-suppression-file "runs/inspect/mi1/suppressions/mySup" -r runs/inspect/mi1

['Created suppression file C:\\Users\\NUC8i7\\Desktop\\EWDemo\\runs\\inspect\\mi1\\suppressions\\mySup.sup from C:\\Users\\NUC8i7\\Desktop\\EWDemo\\runs\\inspect\\mi1. ',
 '  ',
 '1 new problem(s) found ',
 '    1 Missing allocation problem(s) detected ']

In [10]:
%sx inspxe-cl -collect mi1 -r runs/inspect/mi12 mm2.exe

['Memory  ', '0 new problem(s) found ']

In [11]:
!start "" cmd /k inspxe-cl -collect ti2 -r runs/inspect/ti2 deadlock.exe

In [12]:
#create supression file
%sx inspxe-cl.exe \
-r "runs/inspect/ti2" \
-create-suppression-file "runs\inspect\ti2\suppressions\mysup"

['Created suppression file C:\\Users\\NUC8i7\\Desktop\\EWDemo\\runs\\inspect\\ti2\\suppressions\\mysup.sup from C:\\Users\\NUC8i7\\Desktop\\EWDemo\\runs\\inspect\\ti2. ',
 '  ',
 '1 new problem(s) found ',
 '    1 Deadlock problem(s) detected ']

In [13]:
#Reporting: regression status
!inspxe-cl -report status –r runs/inspect/ti2

1 problem(s) found
1 Not investigated
Breakdown by state:
1 New


In [14]:
#Report the observations
%sx inspxe-cl -report observations  -r runs/inspect/ti2/

['  deadlock.exe!0x2cf55: Error X1: P1: Deadlock: Lock owned: Function [Unknown]: Module c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe',
 '  deadlock.exe!0x2cf55: Error X2: P1: Deadlock: Allocation site: Function [Unknown]: Module c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe',
 '  deadlock.exe!0x7749: Error X3: P1: Deadlock: Lock wanted: Function [Unknown]: Module c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe',
 '  deadlock.exe!0x7adb: Error X4: P1: Deadlock: Lock owned: Function [Unknown]: Module c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe',
 '  deadlock.exe!0x7adb: Error X5: P1: Deadlock: Lock wanted: Function [Unknown]: Module c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe',
 '  deadlock.exe!0x7cf3: Error X6: P1: Deadlock: Allocation site: Function [Unknown]: Module c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe']

In [15]:
#Reporting: problem list
!inspxe-cl -report problems –r runs/inspect/ti2

P1: Error: Deadlock: New
 P1.8: Error: Deadlock: New
  deadlock.exe!0x7cf3: Error X12: Allocation site: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  deadlock.exe!0x7adb: Error X13: Lock owned: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  deadlock.exe!0x7adb: Error X14: Lock wanted: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  deadlock.exe!0x2cf55: Error X15: Allocation site: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  deadlock.exe!0x2cf55: Error X16: Lock owned: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  deadlock.exe!0x7749: Error X17: Lock wanted: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
 P1.6: Error: Deadlock: New
  deadlock.exe!0x7cf3: Error X8: Allocation site: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  deadlock.exe!0x7adb: Error X9: Lock owned: Function [Unknown]: Module c:\users\n

In [16]:
#Reporting: extended problem list
!inspxe-cl -report problems -report-all –r runs/inspect/ti2

P1: Error: Deadlock: New
 P1.8: Error: Deadlock: New
  deadlock.exe!0x7cf3: Error X12: Allocation site: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  Stack (1 of 1 instance(s))
  >deadlock.exe![Unknown] - c:\users\nuc8i7\desktop\ewdemo\deadlock.exe:0x7cf3

  deadlock.exe!0x7adb: Error X13: Lock owned: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  Stack (1 of 1 instance(s))
  >deadlock.exe![Unknown] - c:\users\nuc8i7\desktop\ewdemo\deadlock.exe:0x7adb

  deadlock.exe!0x7adb: Error X14: Lock wanted: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  Stack (1 of 1 instance(s))
  >deadlock.exe![Unknown] - c:\users\nuc8i7\desktop\ewdemo\deadlock.exe:0x7adb

  deadlock.exe!0x2cf55: Error X15: Allocation site: Function [Unknown]: Module c:\users\nuc8i7\desktop\ewdemo\deadlock.exe
  Stack (1 of 1 instance(s))
  >deadlock.exe![Unknown] - c:\users\nuc8i7\desktop\ewdemo\deadlock.exe:0x2cf55

  deadlock.exe!0x2cf55: Error 

In [17]:
#Reporting: other CLI options
#XML output
!inspxe-cl -report problems –format=xml -r runs/inspect/ti2

<?xml version="1.0" encoding="UTF-8"?>
<report>
 <pset problem_num="P1" severity_string="Error" diag_type="Deadlock" state_string="New">
  <diag diag_id="8" diag_type="Deadlock" state_string="New" severity_string="Error">
   <obs obs_num="X12" obs_desc="Allocation site" rva="0x7cf3" func_name="[Unknown]" severity_string="Error" fullpath_module_name="c:\users\nuc8i7\desktop\ewdemo\deadlock.exe"></obs>
   <obs obs_num="X13" obs_desc="Lock owned" rva="0x7adb" func_name="[Unknown]" severity_string="Error" fullpath_module_name="c:\users\nuc8i7\desktop\ewdemo\deadlock.exe"></obs>
   <obs obs_num="X14" obs_desc="Lock wanted" rva="0x7adb" func_name="[Unknown]" severity_string="Error" fullpath_module_name="c:\users\nuc8i7\desktop\ewdemo\deadlock.exe"></obs>
   <obs obs_num="X15" obs_desc="Allocation site" rva="0x2cf55" func_name="[Unknown]" severity_string="Error" fullpath_module_name="c:\users\nuc8i7\desktop\ewdemo\deadlock.exe"></obs>
   <obs obs_num="X16" obs_desc="Lock owned" rva="0x2cf55" 

In [18]:
#CSV output
!inspxe-cl -report problems -format csv -csv-delimiter " " -report-output runs/inspect/ti2/observations.csv -r runs/inspect/ti2

In [19]:
import csv
import sys
with open("runs/inspect/ti2/observations.csv", 'r') as csvfile:
		reader = csv.reader(csvfile)
	 
		for row in reader:
			print(row)

['Source "Line" "Severity" "ID" "ID" "Type" "Description" "Function" "Module"']
[]
['deadlock.exe "0x7cf3" "Error" "X12" "P1" "Deadlock" "Allocation site" "[Unknown]" "c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe"']
['deadlock.exe "0x7adb" "Error" "X13" "P1" "Deadlock" "Lock owned" "[Unknown]" "c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe"']
['deadlock.exe "0x7adb" "Error" "X14" "P1" "Deadlock" "Lock wanted" "[Unknown]" "c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe"']
['deadlock.exe "0x2cf55" "Error" "X15" "P1" "Deadlock" "Allocation site" "[Unknown]" "c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe"']
['deadlock.exe "0x2cf55" "Error" "X16" "P1" "Deadlock" "Lock owned" "[Unknown]" "c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe"']
['deadlock.exe "0x7749" "Error" "X17" "P1" "Deadlock" "Lock wanted" "[Unknown]" "c:\\users\\nuc8i7\\desktop\\ewdemo\\deadlock.exe"']
['deadlock.exe "0x7cf3" "Error" "X8" "P1" "Deadlock" "Allocation site" "[Unknown]" "c:\\users\\nuc8i7\\desktop\

In [20]:
#Export full result
!inspxe-cl -export -archive-name runs/inspect/ti2/ti2.inspxez -include-sources -result-dir runs/inspect/ti2

Creating archive runs\inspect\ti2\ti2.inspxez ... 


In [21]:
%%capture
#Launch Intel Inspector 2019 GUI
!"C:\Program Files (x86)\IntelSWTools\Inspector 2019\bin64\inspxe-gui.exe"

In [22]:
#At the end, set the OpenCV tracing flag to zero.
%env OPENCV_TRACE=0

env: OPENCV_TRACE=0
